# Setting

In [20]:
import pandas as pd
import numpy as np
import random
import torch
import pickle
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
from perceiver import crop, patchify, get_patch_coords, tokenize_data, CustomDataset, PerceiverBlock, Perceiver, CombinedModel, ImageDataset

# Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

batch_size = 32

In [21]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정

def seed_worker(worker_id): #데이터로더 난수고정
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

seed_everything(42)
g = torch.Generator()
g.manual_seed(42)
NUM_WORKERS = 4 # 서브프로세스관리자 수. 난수생성과 관련있습니다. 일단은 4로 고정합니다.


# Import things

In [22]:
root_path = '/home/youlee/perceiver/perceiver'

model_path = root_path + '/model/'
loader_path = root_path + '/loader/'

In [23]:
def calculate_text_accuracy(model, data_loader, device):
    model.eval()  # 평가 모드로 전환
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in data_loader:
            # 배치가 dict 형태인 경우 올바른 키 사용
            if isinstance(batch, dict):
                input_ids = batch['input_ids'].to(device)
                labels = batch['labels'].to(device)
                # 모델이 attention_mask를 필요로 한다면 함께 전달
                attention_mask = batch['attention_mask'].to(device)
            else:
                # 만약 tuple/list 형태로 구성되어 있다면, 상황에 맞게 수정
                input_ids, labels = batch[0].to(device), batch[1].to(device)
                attention_mask = None  # 필요 시 추가

            # 모델 호출 (attention_mask가 필요하면 전달)
            if attention_mask is not None:
                outputs = model(input_ids, attention_mask=attention_mask)
            else:
                outputs = model(input_ids)

            # 예: outputs가 각 클래스에 대한 로짓이라면 argmax를 통해 예측 클래스 결정
            predicted = outputs.argmax(dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy


In [32]:
def calculate_image_accuracy(model, data_loader, device):
    model.eval()  # 평가 모드로 전환
    correct = 0
    total = 0
    with torch.no_grad():  # 평가 시에는 기울기 계산하지 않음
        for batch in data_loader:
            inputs, labels = batch

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)  

            predicted = outputs.argmax(dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

In [33]:
for i in range(1, 7):
    model_pos = model_path + f'text_model_{i}.pkl'
    model = torch.load(model_pos, map_location=device)
    model.to(device)
    loader_pos = loader_path + f'text_val_loader_{i}.pkl'
    with open(loader_pos, 'rb') as f:
        loader = pickle.load(f)
    print(f'Task {i-1} model, loader 불러오기 완료')

    val_acc = calculate_text_accuracy(model, loader, device)
    print(f'Task {i-1} 모델의 validation accuracy: {val_acc:.4f}\n')
     

Task 0 model, loader 불러오기 완료


/tmp/ipykernel_3272473/3670095996.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_pos, map_location=device)


Task 0 모델의 validation accuracy: 0.8519

Task 1 model, loader 불러오기 완료
Task 1 모델의 validation accuracy: 0.8171

Task 2 model, loader 불러오기 완료
Task 2 모델의 validation accuracy: 0.8444

Task 3 model, loader 불러오기 완료
Task 3 모델의 validation accuracy: 0.8725

Task 4 model, loader 불러오기 완료
Task 4 모델의 validation accuracy: 0.8705

Task 5 model, loader 불러오기 완료
Task 5 모델의 validation accuracy: 0.7869



In [34]:
for i in range(1, 7):
    model_pos = model_path + f'image_model_{i}.pkl'
    model = torch.load(model_pos, map_location=device)
    model.to(device)
    
    loader_pos = loader_path + f'image_val_loader_{i}.pkl'
    with open(loader_pos, 'rb') as f:
        loader_data = pickle.load(f)
    valid_loader = DataLoader(loader_data, batch_size=batch_size, shuffle=False,
                            num_workers=NUM_WORKERS, worker_init_fn=seed_worker, generator=g)
    print(f'Task {i+5} model, loader 불러오기 완료')

    val_acc = calculate_image_accuracy(model, valid_loader, device)
    print(f'Task {i+5} 모델의 validation accuracy: {val_acc:.4f}\n')

Task 6 model, loader 불러오기 완료


/tmp/ipykernel_3272473/64437764.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_pos, map_location=device)


Task 6 모델의 validation accuracy: 0.7003

Task 7 model, loader 불러오기 완료
Task 7 모델의 validation accuracy: 0.6356

Task 8 model, loader 불러오기 완료
Task 8 모델의 validation accuracy: 0.5809

Task 9 model, loader 불러오기 완료
Task 9 모델의 validation accuracy: 0.6121

Task 10 model, loader 불러오기 완료
Task 10 모델의 validation accuracy: 0.6144

Task 11 model, loader 불러오기 완료
Task 11 모델의 validation accuracy: 0.5353

